In [ ]:
import pandas as pd

In [ ]:
# 1. Load your raw overdose dataset (each row is an overdose case)
df = pd.read_csv("../../reports/deidentified_overdose_201201202408_zips_0311.csv")

# 2. Define the overdose type columns (drug columns)
drug_cols = [
    "Methamphetamine",
    "Heroin",
    "Cocaine",
    "Fentanyl",
    "Alcohol",
    "Prescription.opioids",
    "Any Opioids",
    "Benzodiazepines",
    "Others",
    "Any Drugs",  # This column is assumed to be the overall indicator
]

# 3. Define the demographic columns you want to keep.
# Adjust this list if you want additional fields.
id_vars = ["CaseNumber", "Year", "Age_Bin", "Gender", "Race"]

# 4. Melt the data so that each overdose case appears once per drug type it involved.
#    This creates two new columns: "Overdose_Type" and "DrugIndicator"
df_long = df.melt(
    id_vars=id_vars,
    value_vars=drug_cols,
    var_name="Overdose_Type",
    value_name="DrugIndicator",
)

# 5. Keep only rows where the drug was involved (e.g., DrugIndicator equals 1)
#    (Adjust the condition if your flag is different)
df_long = df_long[df_long["DrugIndicator"] == 1].drop(columns=["DrugIndicator"])

# At this point, df_long is a detailed long table with columns:
# [CaseNumber, Year, Age_Bin, Gender, Race, Overdose_Type]
# where each row represents an overdose case and a specific drug that was involved.

# 6. Create a duplicate of the detailed table with Year set to "All"
df_long_all = df_long.copy()
df_long_all["Year"] = "All"

# 7. Combine the original detailed data with the "All" year rows
df_final = pd.concat([df_long, df_long_all], ignore_index=True)

# (Optional) Create a composite key if needed for filtering in the dashboard.
# For example, if you want to filter by the combination of Year and Overdose_Type:
df_final["YearOverdoseType"] = (
    df_final["Year"].astype(str) + "_" + df_final["Overdose_Type"]
)

df_final["Overdose_Type"] = df_final["Overdose_Type"] + "_Count"

In [ ]:
df_final

In [ ]:
df_final.to_csv("super_long_for_charts_donezo.csv", index=False)